# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam

from scipy.stats import percentileofscore
import time
import math

# Training data

In [ ]:
df = pd.read_parquet('../pipeline/tmp/cities/poz-w-dataset.parquet')
#df = pd.read_parquet('poz-w-dataset.parquet')
df.head(5)

In [ ]:
print(df.size)

In [ ]:
x = df['to_x']
y = df['to_y']

plt.figure(figsize=(6, 5))
plt.hist2d(x, y, bins=100, cmap='viridis')

plt.colorbar(label='Counts')

In [ ]:
X = df[['from_x', 'from_y', 'to_x', 'to_y', 'day_type', 'start']]
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Preprocessing

In [ ]:
class CustomScalerLayer(Layer):
    def __init__(self, city_diameter=25000, **kwargs):
        super(CustomScalerLayer, self).__init__(**kwargs)
        self.city_diameter = city_diameter

    def call(self, inputs):
        # Normalize coordinates
        from_x = (inputs[:, 0] / self.city_diameter) + 0.5
        from_y = (inputs[:, 1] / self.city_diameter) + 0.5
        to_x = (inputs[:, 2] / self.city_diameter) + 0.5
        to_y = (inputs[:, 3] / self.city_diameter) + 0.5

        # Normalize start time
        start = inputs[:, 5] / 86400.0  # 86400 seconds in a day

        # Combine normalized features
        normalized_inputs = tf.stack([from_x, from_y, to_x, to_y, inputs[:, 4], start], axis=1)
        return normalized_inputs

In [ ]:
scaler_layer = CustomScalerLayer()

# Model definition

## architecture

In [ ]:
model = Sequential([
    scaler_layer,
    Dense(64, activation='relu', kernel_initializer=HeNormal()),
    #Dense(64, activation='relu', kernel_initializer=HeNormal(),input_shape=(X_train.shape[1],)),
    #Dense(128, activation='relu', kernel_initializer=HeNormal()),
    #Dense(256, activation='relu', kernel_initializer=HeNormal()),
    #Dense(128, activation='relu', kernel_initializer=HeNormal()),
    Dense(64, activation='relu', kernel_initializer=HeNormal()),
    Dense(16, activation='relu', kernel_initializer=HeNormal()),
    Dense(4, activation='relu', kernel_initializer=HeNormal()),
    Dense(1, activation='exponential')
    #Dense(1, activation='softplus')
])

In [ ]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr * 0.1
    else:
        return lr

In [ ]:
class BatchLearningRateScheduler(Callback):
    def __init__(self, total_batches):
        super(BatchLearningRateScheduler, self).__init__()
        self.total_batches = total_batches
        self.current_epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.current_epoch = epoch

    def on_batch_begin(self, batch, logs=None):
        if self.current_epoch == 0 and batch < self.total_batches:
            lr = tf.keras.backend.get_value(self.model.optimizer.lr)
            ratio = (batch + 1) / self.total_batches
            divisor = ratio if ratio > 0.1 else 0.1
            new_lr = lr * divisor
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
        elif self.current_epoch > 0:
            lr = tf.keras.backend.get_value(self.model.optimizer.lr)
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)

In [ ]:
total_batches = 3000
batch_lr_scheduler = BatchLearningRateScheduler(total_batches)
simple_lr_scheduler = LearningRateScheduler(scheduler)

## loss function

In [ ]:
def asymetric_loss_function(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    tf.debugging.assert_greater(
        tf.reduce_min(y_true), tf.constant(0.0, dtype=tf.float32),
        message="y_true contains zero or negative values!"
    )

    ratio = y_pred / (y_true * 0.9)

    tf.debugging.assert_non_negative(
        ratio,
        message="Ratio is negative!"
    )
    loss = tf.where(ratio <= 1, (1-ratio)**2, 7* tf.math.log(ratio))

    return tf.reduce_mean(loss)

## compile

In [ ]:
model.compile(optimizer='adam', loss=asymetric_loss_function, metrics=['mape'])

# Training

In [ ]:
history = model.fit(X_train, y_train, epochs=8, batch_size=16, validation_split=0.2)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow Select ops.
]
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Model evaluation

## Read tflite model

In [ ]:
with open('model.tflite', 'rb') as f:
    tflite_model = f.read()

interpreter = tf.lite.Interpreter(model_content=tflite_model)

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (1, 6))
interpreter.allocate_tensors()

## predicitons on test set

In [ ]:
y_pred = []

for i in range(X_test.shape[0]):
    # single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float64)
    single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], single_row)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    y_pred.append(output_data[0][0])

y_pred = np.array(y_pred)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

## Visualize set results distribution

In [ ]:
percentage_difference = ((y_pred - y_test) / y_test) * 100

percentile_1 = np.percentile(percentage_difference, 1)
percentile_99 = np.percentile(percentage_difference, 99.9)
max_overestimation = np.max(percentage_difference)
percentile_of_0 = percentileofscore(percentage_difference, 0)

plt.figure(figsize=(10, 6))
sns.histplot(percentage_difference, kde=True, bins=100, color='blue')

plt.axvline(percentile_1, color='green', linestyle='--', linewidth=2, label=f'1st Percentile: {percentile_1:.2f}%')
plt.axvline(percentile_99, color='red', linestyle='--', linewidth=2, label=f'99.9th Percentile: {percentile_99:.2f}%')

plt.text(percentile_1, plt.ylim()[1] * 0.9, f'{percentile_1:.2f}%', color='green', ha='center')
plt.text(percentile_99, plt.ylim()[1] * 0.9, f'{percentile_99:.2f}%', color='red', ha='center')

plt.axvline(max_overestimation, color='purple', linestyle='--', linewidth=2, label=f'Max Overestimation: {max_overestimation:.2f}%')
plt.text(max_overestimation, plt.ylim()[1] * 0.8, f'{max_overestimation:.2f}%', color='purple', ha='center')

plt.xlabel('Percentage Difference (%)')
plt.ylabel('Frequency')
plt.title('Distribution of Percentage Difference Between Predictions and Targets')
plt.legend()
plt.show()

print(f'Maximal Overestimation Value: {max_overestimation:.2f}%')
print(f'Percentile for 0 on x-axis value: {percentile_of_0:.2f}%')

## Forward pass time

In [ ]:
num_iterations = 10000
total_time = 0

for _ in range(num_iterations):
    random_index = np.random.randint(0, X_test.shape[0])
    # single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float64)
    single_row = X_test.iloc[i].to_numpy().reshape(1, -1).astype(np.float32)
    start_time = time.perf_counter()
    interpreter.set_tensor(input_details[0]['index'], single_row)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    end_time = time.perf_counter()
    total_time += (end_time - start_time)

average_forward_pass_time = (total_time / num_iterations) * 1e6
print(f"Average forward pass time for a single row using TensorFlow Lite: {average_forward_pass_time:.6f} microseconds")
print(f"total time for 10000 rows: {total_time*1000} milliseconds")